# ANALYSIS 3

# FINDING THE NEAREST 10 RESTAURANTS TO A GIVEN ADDRESS

### ALL REQUIRED IMPORTS

In [5]:
import string
import csv, sys
from pandas import Series, DataFrame
import pandas as pd
import os
import fnmatch
import json
import nltk
from pprint import pprint
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

#### GETTING THE SYSTEM PATH

In [6]:
directory = os.path.dirname(os.getcwd()) 
path = directory+'/data/'
path
directory

'/Users/sonalichaudhari/Desktop/final'

#### TAKING USER INPUT FOR ADDRESS  

In [8]:
print('STREET NAME AND CITY NAME COUNTRY MANDATORY')
address1 = input("Enter Address: ")
print (address1) 

STREET NAME AND CITY NAME COUNTRY MANDATORY
Enter Address: MUMBAI
MUMBAI


#### Retrieving the complete address with latitude and longitude

In [9]:
# Getting the complete address 

geolocator = Nominatim()
location = geolocator.geocode(address1) #Passing the address which stores user input
print(location.address)
print((location.latitude, location.longitude))

Mumbai, Greater Bombay, Maharashtra, India
(18.9321862, 72.8308337)


In [10]:
# Storing the source latitude and longitude 

s_lat = location.latitude
s_lon = location.longitude

####  STORING OF INTERMEDIATE DATA

In [11]:
DISTANCE={} # to store the  restaurant ids and their distances whose distance is less than 25 miles
temp = [] # to store th data about all the restaurants whose distance is less than 25 miles
for subdir, dirs, files in os.walk(path):
        for file in files:           
            if file.endswith('.json'): # Retrieving only the JSON files
                reqpath = subdir+'/'+file # Path of evry JSON file is stored in reqpath
                
                with open(reqpath) as json_file:
                    g = json.load(json_file)
                    des_lat = g['restaurant']['location']['latitude']  # Destination latitude value
                    des_log = g['restaurant']['location']['longitude'] # Destination longitude value
                    source =(s_lat,s_lon)
                    destination =(des_lat,des_log)
# Finding the distance between the source and destinaltion 
                    d = vincenty(source, destination).miles
                    if d<=25:    # Filtering for distance less than 25(miles)
                        temp.append(g)
                        dd= round(d,4) # Rounding of the float value to 4 decimal places
                        DISTANCE[g['restaurant']['R']['res_id']]= dd
                

In [12]:
# Sorting in ascending order of distance and getting the top 10
S = sorted(DISTANCE.items(), key=lambda x: x[1], reverse=False)[:10]

In [13]:
# Storing the restaurant id of top 10 obtained above in RES_ID list
RES_ID=[]
for k,v in S:
    RES_ID.append(k)
#RES_ID

In [14]:
# Storing the data of restaurant of top 10 obtained above in RES_ID list
RES=[]

for e in temp:
    if (e['restaurant']['R']['res_id'] in RES_ID) == True:
        RES.append(e)

#### STORING THE RES[ ] LIST DATA IN A CSV

In [15]:
with open(directory+'/analysis/ana_3/ANA_3_Part1.csv', 'w') as csvfile:    # Creating CSV
    fieldnames = ['RES_ID', 'Name','CUISINES','USER RATING','VOTES',\
                 'RATING TEXT','ADDRESS','AVG COST for TWO','DISTANCE']#Naming the title Row of the CSV
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for j in range(len(RES)-1):
        for k,v in S: # if restaurant id is present in the list of top 10 nearest restaurants list
            if k == RES[j]['restaurant']['R']['res_id']:
                r = RES[j]['restaurant']['R']['res_id']
                n = RES[j]['restaurant']['name']
                cu= RES[j]['restaurant']['cuisines']
                u = RES[j]['restaurant']['user_rating']['aggregate_rating']
                vo = RES[j]['restaurant']['user_rating']['votes']
                rt= RES[j]['restaurant']['user_rating']['rating_text']
                add= RES[j]['restaurant']['location']['address']
                avg=str(RES[j]['restaurant']['average_cost_for_two'])+temp[j]['restaurant']['currency']
                dis = str(v)+' miles'
                # appending the rows to the csv
                writer.writerow({'RES_ID':r, 'Name':n,'CUISINES':cu,'USER RATING':u,\
                                 'VOTES':vo,'RATING TEXT':rt,'ADDRESS':add,'AVG COST for TWO':avg,'DISTANCE':dis}) 

#### CONVERTING THE CSV INTO DATAFRAME

In [16]:
t = pd.read_csv(directory+'/analysis/ana_3/ANA_3_Part1.csv').reset_index(0,drop=True)
t.head()

,RES_ID,Name,CUISINES,USER RATING,VOTES,RATING TEXT,ADDRESS,AVG COST for TWO,DISTANCE
0,18233317,145 Kala Ghoda,"Fast Food, Beverages, Desserts",4.2,1606,Very Good,"145, Kala Ghoda, Fort, Mumbai",1500Rs.,0.3366 miles
1,49261,Colaba Social,"Mediterranean, American, North Indian, Desserts",4.0,4836,Very Good,"24, Ground Floor, Glen Rose Building, Behind T...",1200Rs.,0.7853 miles
2,18448467,D:OH!,"Fast Food, Italian, Beverages",4.0,148,Very Good,"Ground Floor, Unit 2B, Trade View, Kamala City...",800Rs.,5.0795 miles
3,18313566,Farzi Cafe,Modern Indian,4.3,1240,Very Good,"Kamala Mills, Near Radio Mirchi Office, Lower ...",1500Rs.,4.9106 miles
4,18175468,Grandmama's Café,Cafe,4.1,2098,Very Good,"Shop 5, Near Swaminarayan Mandir, Pritam Estat...",1000Rs.,5.9034 miles
